**One-Hot Encoding: Traduzindo Categorias para Números**

Imagine que você tem uma lista de cores favoritas: "Vermelho", "Azul", "Verde". Para um computador, essas palavras não fazem sentido em cálculos matemáticos. Precisamos de uma forma de representá-las numericamente.

O One-Hot Encoding cria uma nova coluna para cada categoria única em uma característica (feature). Para cada linha de dados, a coluna correspondente à sua categoria recebe o valor 1, e todas as outras colunas de categoria recebem o valor 0. É como acender uma "luz" para a categoria presente e deixar as outras "apagadas".

Por que usar One-Hot Encoding?

* Evita a ordenação artificial: Se tivéssemos "Pequeno", "Médio", "Grande" e os representássemos como 1, 2, 3, um algoritmo poderia erroneamente inferir que "Grande" (3) é "2 vezes maior" que "Pequeno" (1), o que nem sempre faz sentido. O One-Hot Encoding remove essa implicação de ordem.
* Compatibilidade com algoritmos: A maioria dos algoritmos de Machine Learning funciona melhor com dados numéricos.
* Permite modelos entenderem categorias independentes: Cada categoria é tratada como uma entidade separada.


Outras Opções de Codificação Categórica

Embora o One-Hot Encoding seja amplamente usado, há outras opções, e a escolha depende do seu tipo de dado e do problema:

1. Label Encoding (Codificação de Rótulos):

* Como funciona: Atribui um número inteiro único a cada categoria. Por exemplo, "Vermelho" = 0, "Azul" = 1, "Verde" = 2.
* Diferença para One-Hot: Cria uma única coluna numérica.
* Quando usar: Quando a característica categórica tem uma ordem inerente (ordinal). Ex: "Pequeno", "Médio", "Grande". Se não houver ordem, pode levar o modelo a inferir relações numéricas inexistentes (ex: 1 < 2 < 3 para cores, o que não faz sentido).
* Cuidado: Use com cautela em características nominais (sem ordem) para modelos que assumem relações numéricas (como regressão linear, SVMs). Árvores de decisão e Random Forests podem lidar melhor com Label Encoding em dados nominais, mas ainda assim One-Hot é geralmente mais seguro.

2. Target Encoding (Codificação Baseada no Alvo):

* Como funciona: Substitui cada categoria pela média (ou outra estatística) da variável alvo para aquela categoria. Por exemplo, se "Cidade A" tem uma taxa de clique média de 0.7 e "Cidade B" de 0.3, as instâncias de "Cidade A" receberiam 0.7 e as de "Cidade B" receberiam 0.3.
* Diferença para One-Hot/Label: O valor numérico é diretamente influenciado pela relação da categoria com a variável alvo.
* Quando usar: Pode ser muito poderoso para características com muitas categorias únicas (alta cardinalidade) ou quando há uma relação clara entre a categoria e o alvo.
* Cuidado: Altamente propenso a data leakage (vazamento de dados) e overfitting se não for implementado corretamente (ex: calculando a média do alvo usando todo o dataset em vez de apenas o conjunto de treino ou usando validação cruzada).


In [2]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 1.6 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split # <--- Adicione esta linha
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
# Se estiver usando imbalanced-learn ou category_encoders, importe-os também:
# from imblearn.over_sampling import SMOTE
# import category_encoders as ce

print("Iniciando a demonstração didática de Codificação Categórica (One-Hot, Label, Target)...\n")

# --- 1. Geração de Dados Sintéticos ---
np.random.seed(42)
num_clientes = 100

data = {
    'Gênero': np.random.choice(['Masculino', 'Feminino', 'Outro'], num_clientes, p=[0.45, 0.50, 0.05]),
    'Nível_Satisfacao': np.random.choice(['Baixo', 'Médio', 'Alto'], num_clientes, p=[0.2, 0.5, 0.3]),
    'Cidade': np.random.choice([f'Cidade_{i}' for i in range(1, 15)], num_clientes, p=np.random.dirichlet(np.ones(14), size=1)[0]), # Simula muitas cidades
    'Comprou': np.random.randint(0, 2, num_clientes) # Variável alvo (0 ou 1)
}
df = pd.DataFrame(data)

print("PASSO 1: Geramos um conjunto de dados sintético com características categóricas.")
print(f"  Amostra dos dados originais:\n{df.head()}\n")
print(f"  Contagem de categorias para 'Gênero': {df['Gênero'].value_counts().to_dict()}")
print(f"  Contagem de categorias para 'Nível_Satisfacao': {df['Nível_Satisfacao'].value_counts().to_dict()}")
print(f"  Número de cidades únicas: {df['Cidade'].nunique()}\n")
print("---------------------------------------------------\n")

# --- 2. Aplicação de ONE-HOT ENCODING ---
print("PASSO 2: Aplicando ONE-HOT ENCODING para 'Gênero'.")
print("  -> Cria novas colunas para cada categoria, com 1 para a categoria presente e 0 para as outras.")
print("  -> Ideal para características NOMINAIS (sem ordem, como cores, gêneros).")

# Criamos um OneHotEncoder
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False) # sparse_output=False para ter um array denso

# Ajusta (aprende as categorias) e transforma a coluna 'Gênero'
gender_encoded = one_hot_encoder.fit_transform(df[['Gênero']])

# Cria um DataFrame com as novas colunas e seus nomes
gender_df_ohe = pd.DataFrame(gender_encoded, columns=one_hot_encoder.get_feature_names_out(['Gênero']))

print(f"  Resultado do One-Hot Encoding para 'Gênero':\n{gender_df_ohe.head()}\n")
print("  As colunas originais de 'Gênero' são substituídas por 'Gênero_Feminino', 'Gênero_Masculino', etc.\n")
print("---------------------------------------------------\n")

# --- 3. Aplicação de LABEL ENCODING ---
print("PASSO 3: Aplicando LABEL ENCODING para 'Nível_Satisfacao'.")
print("  -> Atribui um número inteiro único para cada categoria em UMA ÚNICA coluna.")
print("  -> Ideal para características ORDINAIS (com ordem, como Pequeno, Médio, Grande).")

# Criamos um LabelEncoder
label_encoder = LabelEncoder()

# Definimos a ordem para garantir que 'Baixo' seja 0, 'Médio' 1, 'Alto' 2
satisfaction_mapping = {'Baixo': 0, 'Médio': 1, 'Alto': 2}
df['Nível_Satisfacao_Encoded'] = df['Nível_Satisfacao'].map(satisfaction_mapping)

print(f"  Resultado do Label Encoding para 'Nível_Satisfacao':\n{df[['Nível_Satisfacao', 'Nível_Satisfacao_Encoded']].head()}\n")
print("  'Baixo' se tornou 0, 'Médio' 1, 'Alto' 2, mantendo a ordem.\n")
print("---------------------------------------------------\n")

# --- 4. Aplicação de TARGET ENCODING ---
print("PASSO 4: Aplicando TARGET ENCODING para 'Cidade'.")
print("  -> Substitui a categoria pela média da variável alvo ('Comprou') para aquela categoria.")
print("  -> Útil para características com muitas categorias (alta cardinalidade).")

# Para Target Encoding, precisamos separar treino e teste para EVITAR VAZAMENTO DE DADOS (Data Leakage)
# Vazamento de dados ocorre se usarmos a informação do alvo do conjunto de teste para codificar.
X_train, X_test, y_train, y_test = train_test_split(
    df[['Gênero', 'Nível_Satisfacao', 'Cidade']], df['Comprou'], test_size=0.3, random_state=42, stratify=df['Comprou']
)

# Criamos um TargetEncoder
# O cols=['Cidade'] indica qual coluna queremos encodar
# A 'smoothing' ajuda a evitar que categorias raras tenham valores extremos
target_encoder = ce.TargetEncoder(cols=['Cidade'], smoothing=0.1)

# Fit (aprende as médias) APENAS no conjunto de TREINO
target_encoder.fit(X_train, y_train)

# Transforma o treino e o teste
X_train_target_encoded = target_encoder.transform(X_train)
X_test_target_encoded = target_encoder.transform(X_test)

print(f"  Amostra do 'Cidade' no Treino após Target Encoding:\n{X_train_target_encoded[['Cidade']].head()}\n")
print(f"  Amostra do 'Cidade' no Teste após Target Encoding:\n{X_test_target_encoded[['Cidade']].head()}\n")
print("  Cada cidade agora é representada pela média de 'Comprou' para aquela cidade no treino.\n")
print("---------------------------------------------------\n")


# --- 5. Visualização Comparativa ---
print("PASSO 5: Visualizamos como as colunas são alteradas por cada método.")

# Criando um DataFrame para visualização consolidada
# Não é um dataset para modelo, apenas para mostrar as transformações
df_viz = df.copy()

# Adiciona colunas One-Hot
df_viz = pd.concat([df_viz.drop(columns=['Gênero']), gender_df_ohe], axis=1)

print("\n--- RESUMO FINAL ---")
print("Observe as transformações nas colunas:")
print("\n**ONE-HOT ENCODING:**")
print("- Transforma uma ÚNICA coluna categórica (ex: 'Gênero') em MÚLTIPLAS colunas binárias (0 ou 1).")
print("- `Gênero_Feminino`, `Gênero_Masculino`, `Gênero_Outro` - apenas uma é 1 por linha.")
print("- **Use para:** Categorias sem ordem (nominais).")
print("- **Benefício:** Evita que o modelo crie uma ordem falsa entre as categorias.")

print("\n**LABEL ENCODING:**")
print("- Transforma uma ÚNICA coluna categórica em UMA ÚNICA coluna numérica inteira.")
print("- `Nível_Satisfacao_Encoded` - 'Baixo'=0, 'Médio'=1, 'Alto'=2.")
print("- **Use para:** Categorias com ordem (ordinais).")
print("- **Cuidado:** Se não houver ordem, modelos como Regressão Linear podem interpretar os números como magnitudes e criar relações incorretas.")

print("\n**TARGET ENCODING:**")
print("- Transforma uma ÚNICA coluna categórica em UMA ÚNICA coluna numérica float.")
print("- `Cidade` é substituída pela média de 'Comprou' para cada cidade.")
print("- **Use para:** Categorias com muitas opções (alta cardinalidade) ou quando a relação com o alvo é importante.")
print("- **Cuidado:** Exige tratamento cuidadoso para evitar vazamento de dados (data leakage) e overfitting, pois usa informações do alvo. Sempre ajuste (fit) no treino e transforme no teste.")

print("\nA escolha do método depende da natureza da sua característica categórica (nominal, ordinal) e do seu objetivo com o modelo.")

Iniciando a demonstração didática de Codificação Categórica (One-Hot, Label, Target)...

PASSO 1: Geramos um conjunto de dados sintético com características categóricas.
  Amostra dos dados originais:
      Gênero Nível_Satisfacao     Cidade  Comprou
0  Masculino            Baixo  Cidade_11        0
1      Outro            Médio   Cidade_4        0
2   Feminino            Médio   Cidade_4        0
3   Feminino            Médio  Cidade_12        1
4  Masculino             Alto  Cidade_11        0

  Contagem de categorias para 'Gênero': {'Masculino': 49, 'Feminino': 46, 'Outro': 5}
  Contagem de categorias para 'Nível_Satisfacao': {'Médio': 52, 'Alto': 30, 'Baixo': 18}
  Número de cidades únicas: 12

---------------------------------------------------

PASSO 2: Aplicando ONE-HOT ENCODING para 'Gênero'.
  -> Cria novas colunas para cada categoria, com 1 para a categoria presente e 0 para as outras.
  -> Ideal para características NOMINAIS (sem ordem, como cores, gêneros).
  Resultado do 